In [5]:
import glob
import pandas as pd
import os
from nltk.tree import *
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
import re
import string 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
def chunking(text):
    
    chunks=[]
    chunks=(list(ne_chunk(pos_tag(word_tokenize(text)))))
    for i in chunks:
        
        if type(i)==Tree:
            if i.label() == "GPE":
                j = i.leaves()
                if len(j)>1:
                    gpe = "_".join([term for term,pos in j])
                    text = re.sub(rf'{j[1][0]}',gpe,text, flags=re.MULTILINE)           
                    text = re.sub(rf'\b{j[0][0]}\b',"",text, flags=re.MULTILINE) 
            if i.label()=="PERSON":
                for term,pog in i.leaves():
                    text = re.sub(re.escape(term),"",text, flags=re.MULTILINE)
    return text

def subject(text):
    text = re.sub(r"Re:"," ",text,flags=re.MULTILINE)
    line = re.findall(r'^Subject.*$',text, re.MULTILINE)
    sub = line[0]
    #sub = sub[8:]   #Truncate Subject
    for i in string.punctuation:   #remove all the non-alphanumeric
        sub = sub.replace(i," ")
    sub = re.sub(r"Re:"," ",sub, re.MULTILINE) #removing Re
    sub = sub.lower()  #lower-casing
    sub = re.sub(r'Subject'," ",sub,flags=re.IGNORECASE)  #removing subject
    sub=re.sub(r"\s+", " ", sub)#removing space
    sub=re.sub(r"^\s+", "", sub)#removing leading space
    return sub

def email_preprocess(text):
    emails = re.findall(r"@[A-Za-z0-9\.\-+_]+\.[A-Za-z]+", text)
    all_IDs=[]
    for email in emails:
        ID = email.split('@')[-1].split('.')
        if 'com' in ID:
            ID.remove('com')
        if 'Com' in ID:
            ID.remove('Com')
        rm = [x for x in ID if len(x)<=2 ]
        new_ID = [x for x in ID if x not in rm]
        all_IDs.extend(new_ID)
    final_IDs = []
    [final_IDs.append(x) for x in all_IDs if x not in final_IDs]
    #print(final_IDs)  
    return final_IDs

def Text_preprocessing(file):
    f1 = open(file,"r+")
    txt_class = file.split("_")[0]
        
    txt = f1.read()
        
    #processing emails    
    email = email_preprocess(txt)
    if email is not None:
        email = " ".join(email)
        
    #remove emails
    txt = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)",' ',txt)
    
    #store subject and remove it from text  
    sbj = subject(txt)
    txt = re.sub(r'Subject.*$'," ",txt,flags=re.MULTILINE)
    sbj = re.sub('\W+',' ', sbj)
    
    
    #remove sentences starting with From and Write to
    txt = re.sub(r"Write to:.*$","",txt, flags=re.MULTILINE)           
    txt = re.sub(r"From:.*$","",txt, flags=re.MULTILINE)               
    txt = re.sub(r"or:"," ",txt,flags=re.MULTILINE)
    
    #Task-6,7,8
    txt = re.sub(r"<.*>","",txt, flags=re.MULTILINE) 
    txt = re.sub(r"\(.*\)","",txt,flags=re.MULTILINE)
    txt = re.sub(r"\([^()]*\)", "", txt) 
    txt = re.sub(r"[\n\t\-\\\/]"," ",txt, flags=re.MULTILINE)
    
    #Task-9
    txt = re.sub(r'\w+:\s?','',txt)
    txt = re.sub('\s+',' ',txt)
    #txt = re.sub('^>+','',txt)
    txt = re.sub('\W+',' ', txt )  
    
 
    #Task -10
    txt = re.sub(r"won\'t", "will not", txt)
    txt = re.sub(r"can\'t", "can not", txt)

    # general
    txt = re.sub(r"n\'t", " not", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'s", " is", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'t", " not", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'m", " am", txt)
    
    #Task-11
    txt = chunking(txt)
                
        #removing numbers
     ## Delete Number 
        
    txt = re.sub('[0-9\n]',' ',txt)
        
    
    #14
    txt = re.sub(r"\b_([a-zA-z]+)_\b",r"\1",txt)
    
    txt = re.sub(r"\b_([a-zA-z]+)\b",r"\1",txt) #replace_word to word
    txt = re.sub(r"\b([a-zA-z]+)_\b",r"\1",txt)
    
    #15
    txt = re.sub(r"\b[a-zA-Z]{1}_([a-zA-Z]+)",r"\1",txt) #d_berlin to berlin
    txt = re.sub(r"\b[a-zA-Z]{2}_([a-zA-Z]+)",r"\1",txt) #mr_cat to cat
    
    #16
    txt = txt.lower()      #lower case
    txt = re.sub(r'\b\w{1,2}\b'," ",txt) #remove words <2
    txt = re.sub(r"\b\w{15,}\b"," ",txt) #remove words >15

    txt = re.sub(r"[^a-zA-Z_]"," ",txt)  #keep only alphabets and _
        
    #17                                        
    txt = re.sub(r" {2,}", " ", txt, flags=re.MULTILINE) # REMOVE THE EXTRA SPACES
    
    return email, sbj, txt, txt_class
    

In [19]:
files_folder = r"E:/projectss/CNN_with_textdata/documents/"
os.chdir(files_folder)
all_emails = []
all_subjects = []
all_text = []
all_classes = []
i=0
for file in tqdm(os.listdir()): 
    
    preprocessed_emails,subjects,text,txt_class = (Text_preprocessing(file))
    all_emails.append(preprocessed_emails)
    all_subjects.append(subjects)
    all_text.append(text)
    all_classes.append(txt_class)

100%|████████████████████████████████████████████████████████████████████████████| 18828/18828 [23:16<00:00, 13.48it/s]


In [25]:
df = pd.DataFrame({'class':all_classes,'preprocessed_text':all_text,'preprocessed_subject':all_subjects,'preprocessed_emails':all_emails})
#print(df.tail())
df.to_csv('E:/projectss/CNN_with_textdata/data.csv')